# gRPC in Python

Let us replicate the user management service, but this time using gRPC instead of a REST API

## Message and method definition in the Protocol Buffer file

In [ ]:
// Protocol Buffer revision specification
syntax = "proto3";

// Package declaration to prevent naming conflicts between different projects
package people;

// Resource message definition, including each field specification and their respective 
// "tag" used in the binary encoding.
message User {
  string id = 1;
  string name = 2;
  string email = 3;
}

// Message definition to list all resources
message UserList {
  repeated User user = 1;
}

// Empty message definition
message Empty {}

// Message definition for creating a new resource (request)
message CreateUserRequest {
  string name = 1;
  string email = 2;
}

// Message definition for creating a new resource (response)
message CreateUserResponse {
  string id = 1;
  string message = 2;
}

// Message definition to retrieve a given resource
message UserIdRequest {
  string id = 1;
}

// Message definition for updating/deleting a given resource (response)
message UpdateUserResponse {
  string message = 1;
}

// RPC service definition, including the relevant interactions
service UserService {
  // Method to retrieve all resources
  rpc GetAllUsers (Empty) returns (UserList);
  // Method to create a new resource
  rpc CreateUser (CreateUserRequest) returns (CreateUserResponse);
  // Method to retrieve a specified resource
  rpc GetUserById (UserIdRequest) returns (User);
  // Method to update a specified resource
  rpc UpdateUser (User) returns (UpdateUserResponse);
  // Method to delete a specified resource
  rpc DeleteUserById (UserIdRequest) returns (UpdateUserResponse);
}


If needed, install the `grpcio-tools`:

`pip install grpcio-tools`

Then run the following command to generate the required Python gRPC files, starting fom the protobuf file above:

`python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. people.proto`

This will create two files:
  - `people_pb2.py` defining message classes in Python
  - `people_pb2_grpc.py` defining the service stub

To verify how gRPC encodes and serializes the messages defined in the protobuf file, retrieve from `people_pb2.py` the serialized protobuf data in binary format, assigned to `DESCRIPTOR`:

In [4]:
serialized_protobuf = b'\n\x0cpeople.proto\x12\x06people\"/\n\x04User\x12\n\n\x02id\x18\x01 \x01(\t\x12\x0c\n\x04name\x18\x02 \x01(\t\x12\r\n\x05\x65mail\x18\x03 \x01(\t\"&\n\x08UserList\x12\x1a\n\x04user\x18\x01 \x03(\x0b\x32\x0c.people.User\"\x07\n\x05\x45mpty\"0\n\x11\x43reateUserRequest\x12\x0c\n\x04name\x18\x01 \x01(\t\x12\r\n\x05\x65mail\x18\x02 \x01(\t\"1\n\x12\x43reateUserResponse\x12\n\n\x02id\x18\x01 \x01(\t\x12\x0f\n\x07message\x18\x02 \x01(\t\"\x1b\n\rUserIdRequest\x12\n\n\x02id\x18\x01 \x01(\t\"%\n\x12UpdateUserResponse\x12\x0f\n\x07message\x18\x01 \x01(\t2\xb3\x02\n\x0bUserService\x12.\n\x0bGetAllUsers\x12\r.people.Empty\x1a\x10.people.UserList\x12\x43\n\nCreateUser\x12\x19.people.CreateUserRequest\x1a\x1a.people.CreateUserResponse\x12\x32\n\x0bGetUserById\x12\x15.people.UserIdRequest\x1a\x0c.people.User\x12\x36\n\nUpdateUser\x12\x0c.people.User\x1a\x1a.people.UpdateUserResponse\x12\x43\n\x0e\x44\x65leteUserById\x12\x15.people.UserIdRequest\x1a\x1a.people.UpdateUserResponseb\x06proto3'

Then decode it:

In [ ]:
from google.protobuf import descriptor_pb2

# Create a FileDescriptorProto object
file_descriptor_proto = descriptor_pb2.FileDescriptorProto()
file_descriptor_proto.ParseFromString(serialized_protobuf)

# Print the decoded .proto definition
print(file_descriptor_proto)


## gRPC Server

Now let us write the code that must be run at the server side

In [ ]:
import grpc
import uuid
import people_pb2
import people_pb2_grpc
from concurrent import futures

class UserService(people_pb2_grpc.UserServiceServicer):
    """
    Implementation of the gRPC UserService.
    This service manages user creation, retrieval, modification, and deletion.
    """
    
    def __init__(self):
        # Dictionary to store user data with unique user_id as key
        self.users = {}  # Example: {"user_id": {"name": "Alice", "email": "alice@example.com"}}

    def GetAllUsers(self, request, context):
        """
        Retrieve all users in the system.
        Returns a UserList message containing all registered users.
        """
        users_list = [
            people_pb2.User(id=user_id, name=user["name"], email=user["email"])
            for user_id, user in self.users.items()
        ]
        return people_pb2.UserList(user=users_list)

    def CreateUser(self, request, context):
        """
        Create a new user with a unique UUID, name, and email.
        Returns a UserResponse containing the new user's ID and a success message.
        """
        user_id = str(uuid.uuid4())  # Generate a unique user ID
        self.users[user_id] = {"name": request.name, "email": request.email}
        
        return people_pb2.CreateUserResponse(
            id=user_id,
            message="User created successfully"
        )

    def GetUserById(self, request, context):
        """
        Retrieve a user's details by their unique ID.
        If the user is found, return their information as a User message.
        If the user is not found, return a NOT_FOUND gRPC status.
        """
        user = self.users.get(request.id)
        if not user:
            context.set_code(grpc.StatusCode.NOT_FOUND)
            context.set_details(f"User with ID {request.id} not found")
            return people_pb2.User()  # Return empty user object if not found

        return people_pb2.User(
            id=request.id,
            name=user["name"],
            email=user["email"]
        )
    
    def UpdateUser(self, request, context):
        """
        Update an existing user's details.
        Only updates fields that are provided in the request (name/email).
        If the user does not exist, returns a NOT_FOUND gRPC status.
        """
        user = self.users.get(request.id)
        if not user:
            context.set_code(grpc.StatusCode.NOT_FOUND)
            context.set_details(f"User with ID {request.id} not found")
            return people_pb2.UpdateUserResponse(message="User not found")

        # Update user details if provided
        if request.name:
            user["name"] = request.name
        if request.email:
            user["email"] = request.email

        return people_pb2.UpdateUserResponse(
            message=f"User with ID {request.id} updated successfully"
        )

    def DeleteUserById(self, request, context):
        """
        Delete a user by their unique ID.
        If the user exists, they are removed, and a confirmation message is returned.
        If the user does not exist, a NOT_FOUND gRPC status is returned.
        """
        if request.id in self.users:
            del self.users[request.id]
            return people_pb2.UpdateUserResponse(
                message=f"User with ID {request.id} deleted successfully"
            )

        # User not found, return NOT_FOUND status
        context.set_code(grpc.StatusCode.NOT_FOUND)
        context.set_details(f"User with ID {request.id} not found")
        return people_pb2.UpdateUserResponse(message="")


def serve():
    """
    Start the gRPC server and listen for client requests.
    The server runs on port 50051 and handles multiple requests using a thread pool.
    """
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))  # Multi-threaded server

    # Add a servicer to the server, with the RPC methods specified in the Protocol Buffer file
    people_pb2_grpc.add_UserServiceServicer_to_server(UserService(), server)
    
    # Start listening on all interfaces on port 50051
    server.add_insecure_port("[::]:50051")
    server.start()
    
    try:
        print("User Management gRPC Server is running on port 50051...")
        server.wait_for_termination()

    except KeyboardInterrupt:
        print("The server is terminating...")


if __name__ == "__main__":
    serve() # Run the server when executed


## gRPC Client

And finally, let us write a simple interactive client program (to be run from the CLI)

In [ ]:
import people_pb2
import people_pb2_grpc
import argparse

# This is where we can configure which gRPC server we want to connect to
from people_conf import channel

def run_client(action, user_name, user_email, user_id):
    
    stub = people_pb2_grpc.UserServiceStub(channel)

    if action == "add":
        # Create a user
        response = stub.CreateUser(people_pb2.CreateUserRequest(name=user_name, email=user_email))
        user_id = response.id
        print(f"{response.message} - ID = {user_id}")

    elif action == "get":
        if user_id:
            # Retrieve specified User
            user = stub.GetUserById(people_pb2.UserIdRequest(id=user_id))
            if user.id:
                print(f"User: ID={user.id}, Name={user.name}, Email={user.email}")
        else:
            # Retrieve all users
            user_list = stub.GetAllUsers(people_pb2.Empty())
            print(f"List of retrieved users:\n")
            for user in user_list.user:
                print(f"ID: {user.id}, Name: {user.name}, Email: {user.email}")
            print(f"\n{len(user_list.user)} users retrieved")

    elif action == "mod":
        # Update user details
        update_response = stub.UpdateUser(people_pb2.User(id=user_id, name=user_name, email=user_email))
        print(update_response.message)

    elif action == "del":
        # Delete user
        delete_response = stub.DeleteUserById(people_pb2.UserIdRequest(id=user_id))
        print(delete_response.message)


if __name__ == "__main__":

    parser = argparse.ArgumentParser(description="Simple gRPC-based user management interactive script.")
    parser.add_argument('action', metavar='ACTION', type=str, choices=['add', 'del', 'mod', 'get'], help='Action to be executed on the specified user')
    parser.add_argument('-n', '--name', metavar='NAME', type=str, help='User name')
    parser.add_argument('-e', '--email', metavar='EMAIL', type=str, help='User email')
    parser.add_argument('-i', '--id', metavar='ID', type=str, help='User ID')
    args = parser.parse_args()
    if args.action == "add" and (args.name is None or args.email is None):
        parser.error("Both name and email are required when adding a new user.")
    elif args.action == "del" and args.id is None:
        parser.error("User ID is required when deleting a user.")
    elif args.action == "mod" and args.id is None:
        parser.error("User ID is required when updating a user.")

    run_client(args.action, args.name, args.email, args.id)
